# <center><span style="color:#2C786C">프로젝트 : 추론하는 인공지능</span></center>

## <span style="color:#F7B400">목표</span>
>* **<span style="color:#2C786C">두 개 이상의 입력을 받는 모델을 설계해본다.</span>**
>* **<span style="color:#2C786C">메모리 구조를 사용하는 메모리 네트워크에 대해서 이해한다.</span>**
>* **<span style="color:#2C786C">End-to-End Memory Network를 통해 bAbI QA task 를 수행하는 프로젝트를 진행한다.</span>**

## <span style="color:#F7B400">순서</span>
> **<span style="color:#2C786C">Step 1. 토크나이저 변경하기 (매우 중요!!!)</span>**<br>
**<span style="color:#2C786C">Step 2. 형태소 분석 후 불용어 처리하기</span>**<br>
**<span style="color:#2C786C">Step 3. 한국어에서의 모델 정확도 확인해보기</span>**<br>
**<span style="color:#2C786C">Step 4. 정리</span>**<br>
**<span style="color:#2C786C">Step 5. 루브릭 평가</span>**<br>
**<span style="color:#2C786C">Step 6. 회고</span>**

### <span style="color:#926DD6">참고</span>
>* []()
>* []()

### <span style="color:#926DD6">개념</span>
>* 
>>*
>>*

>* 
>>*
>>*

그럼 시작해 볼까요?
아래의 데이터셋은 bAbI 데이터셋을 저자가 한국어로 변환한 한국어 버전의 bAbI 데이터셋입니다.

#### <span style="color:#124445">✓ 데이터 다운로드(로컬 유저용)</span>
* [e-28-korean.zip ](https://aiffelstaticprd.blob.core.windows.net/media/documents/e-28-korean.zip) 👉 압축을 해제해서 사용해주세요.

#### <span style="color:#124445">✓ 데이터 다운로드(☁️클라우드 유저용)</span>

    $ ln -s ~/data ~/aiffel/babi_memory_net/data

#### <span style="color:#124445">✓ [데이터셋 소개]</span>

    # 데이터셋의 형태는 아래와 같이 영어 데이터셋과 동일합니다.
    
    1 은경이는 복도로 가버렸습니다.
    2 필웅이는 화장실로 뛰어갔습니다.
    3 은경이는 어디야?     복도  1
    4 수종이는 화장실로 복귀했습니다.
    5 은경이는 침실로 갔습니다.
    6 필웅이는 어디야?     화장실 2
    7 은경이는 복도로 이동했습니다.
    8 경임이는 부엌으로 뛰어갔습니다.
    9 경임이는 어디야?     부엌  8
    10 경임이는 복도로 가버렸습니다.
    11 은경이는 정원으로 이동했습니다.
    12 경임이는 어디야?     복도  10
    13 경임이는 화장실로 복귀했습니다.
    14 경임이는 부엌으로 갔습니다.
    15 경임이는 어디야?     부엌  14

이번 프로젝트에서는 위 한국어 버전의 데이터셋에 대해서 동작하는 QA 모델을 만들어보겠습니다.

> **<span style="color:#D43838">주의❗</span>** 기존 케라스 공식 문서의 babi_rnn, babi_memn 구현은 <span style="color:#D43838">파이썬 3.6을 기준</span>으로 하고 있으며, 파이썬 3.7에서는 정상동작 하지 않을 수 있습니다. 실습 시 참고하세요!

## <span style="color:#2C786C">Step 1. 토크나이저 변경하기 (매우 중요!!!)</span>
---
영어권 언어는 띄어쓰기만해도 단어들이 잘 분리되지만, 한국어는 그렇지 않다고 앞에서 몇 차례 언급했었죠? 한국어 데이터를 사용하여 모델을 구현하는 것만큼 이번에는 형태소 분석기를 사용해서 단어 토큰화를 해보겠습니다.

그런데 형태소 분석기를 사용할 때, 이런 상황에 봉착한다면 어떻게 해야할까요?

    형태소 분석 입력 : '은경이는 사무실로 갔습니다.'
    형태소 분석 결과 : ['은', '경이', '는', '사무실', '로', '갔습니다', '.']

사실 위 문장에서 '은경이'는 사람 이름이므로 제대로 된 결과를 얻기 위해서는 '은', '경이'와 같이 글자가 분리되는 것이 아니라 '은경이' 또는 최소한 '은경'이라는 단어 토큰을 얻어야만 합니다.

이런 경우에는 형태소 분석기에 사용자 사전을 추가해줄 수 있습니다.

'은경이'는 하나의 단어이기 때문에 분리하지말라고 형태소 분석기에 알려주는 것이죠.

사용자 사전을 추가하는 방법은 형태소 분석기마다 다소 다른데, 생각보다 복잡한 경우도 많습니다. 이번 실습에서는 Customized Konlpy라는 사용자 사전 추가가 매우 쉬운 패키지를 사용합니다.

### <span style="color:#124445">Customized Konlpy 설치 방법</span>

#### <span style="color:#124445">✓ 로컬유저용</span>
설치 방법과 사용 방법에 대한 자세한 설명은 아래의 링크를 참고하세요!

* [Customized Konlpy 사용하기](https://inspiringpeople.github.io/data%20analysis/ckonlpy/)

#### <span style="color:#124445">✓ ☁️클라우드 유저용</span>
아래 명령어를 Cloud shell에 입력하세요.

    $ pip install customized_konlpy

가령, 트위터라는 형태소 분석기를 사용한다고 하였을 때, '은경이'라는 단어를 사용자 사전에 추가하고 나서 문장을 형태소 분석하려면 어떻게 해야 할까요?

In [2]:
# 예시 코드
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.add_dictionary('은경이', 'Noun')
twitter.morphs('은경이는 사무실로 갔습니다.')

ModuleNotFoundError: No module named 'ckonlpy'

## <span style="color:#2C786C">Step 2. 추출된 결과로 embedding model 만들기</span>
---
형태소 분석기로 문자를 분석해보면 '는', '으로' 등 분석에 크게 도움이 되지 않을 것 같은 토큰들이 나올 거에요. 처음에는 이 토큰들도 그대로 사용해서 모델을 구현해보고, 두 번째 구현에서는 이 토큰들을 전처리 과정에서 불용어 처리하여 제외해 봅시다.

## <span style="color:#2C786C">Step 3. target, attribute 단어 셋 만들기</span>
---
앞서 만든 메모리 네트워크는 영어권 데이터에서는 보편적으로 약 96% 이상의 높은 성능을 보입니다. 하이퍼파라미터를 잘 조정하면 이보다 더 높은 정확도가 나오기도 합니다. 그렇다면 메모리 네트워크가 한국어에서도 영어만큼 잘 동작할까요? 직접 확인해 봅시다.

## <span style="color:#2C786C">Step 4. 정리</span>
---

## <span style="color:#2C786C">Step 5. 루브릭 평가</span>
---

|평가문항|상세기준|
|-------|--------|
|1. 한국어의 특성에 알맞게 전처리가 진행되었다.|한국어 특성에 따른 토큰화, 임베딩을 거쳐 데이터셋이 적절히 구성되었다.|
|2. 메모리 네트워크가 정상적으로 구현되어 학습이 안정적으로 진행되었다.|validation loss가 안정적으로 수렴하는 것을 확인하고 이를 시각화하였다.|
|3. 메모리 네트워크를 통해 한국어 bAbI 태스크의 높은 정확도를 달성하였다.|추론 태스크의 테스트 정확도가 90% 이상 달성하였다.|

## <span style="color:#2C786C">Step 6. 회고</span>
---